# 🔍 Research Subroutine Demo

Probe one verification check with either OpenAI hosted search or Brave Search API.

## 🔎 Sections
- ⚙️ Setup + auto-reload
- 🧠 Check payload
- 🌐 Provider config (`openai` vs `brave`)
- 🚀 Run research agent
- 🧪 Quick tweaks

## ⚙️ Setup + auto-reload

In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import json
import os
import sys

ROOT = Path.cwd().resolve().parent if Path.cwd().name == "notebooks" else Path.cwd().resolve()
SRC = ROOT / "src"
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))

print(f"Root: {ROOT}")
print(f"Using src path: {SRC}")
print(f"OPENAI_API_KEY set: {bool(os.getenv('OPENAI_API_KEY'))}")
print(f"BRAVE_SEARCH_API_KEY set: {bool(os.getenv('BRAVE_SEARCH_API_KEY'))}")

Root: /Users/bertik/PhD/facticli
Using src path: /Users/bertik/PhD/facticli/src
OPENAI_API_KEY set: True
BRAVE_SEARCH_API_KEY set: False


## 🧠 Check payload

In [ ]:
from facticli.types import VerificationCheck

claim = "The Eiffel Tower was built in 1889 for the World's Fair."
# claim = "The first iPhone launched in 2008."
# claim = "Pluto is currently classified as a planet by the IAU."

check = VerificationCheck(
    aspect_id="timeline_1",
    question="Was the Eiffel Tower completed in 1889 and linked to the World's Fair?",
    rationale="Date and event linkage are core to this claim.",
    search_queries=[
        "Eiffel Tower completion year",
        "Eiffel Tower 1889 Exposition Universelle",
    ],
)

payload = {
    "claim": claim,
    "check": check.model_dump(),
    "requirements": {
        "min_sources": 2,
        "must_use_search_tool": True,
    },
}

payload

## 🌐 Provider config (`openai` vs `brave`)

In [ ]:
model = os.getenv("FACTICLI_MODEL", "gpt-4.1-mini")
search_provider = os.getenv("FACTICLI_SEARCH_PROVIDER", "openai")
# search_provider = "brave"

search_context_size = "high"

if search_provider == "brave":
    assert os.getenv("BRAVE_SEARCH_API_KEY"), "Set BRAVE_SEARCH_API_KEY for brave mode."

print(model, search_provider, search_context_size)

## 🚀 Run research agent

In [ ]:
from agents import Runner
from facticli.agents import build_research_agent
from facticli.types import AspectFinding

research_agent = build_research_agent(
    model=model,
    search_context_size=search_context_size,
    search_provider=search_provider,
)

result = await Runner.run(research_agent, json.dumps(payload, indent=2))
finding = result.final_output_as(AspectFinding, raise_if_incorrect_type=True)
finding.model_dump()

## 🧪 Quick tweaks

- Redefine `claim` and `check` above.
- Flip provider with `search_provider = "brave"`.
- Tune `src/facticli/prompts/research.md` and rerun to compare source grounding.